DATA PROCESSING (TIỀN XỬ LÝ DỮ LIỆU)

In [ ]:
import pandas as pd
import numpy as np
import re
import unicodedata
from typing import List, Tuple, Dict
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# ============================================================================
# PHẦN 1: CÁC HÀM TIỀN XỬ LÝ
# ============================================================================

def normalize_unicode(text: str) -> str:
    if not isinstance(text, str):
        return ""
    return unicodedata.normalize('NFC', text)


def minimal_clean_text(text: str) -> str:
    if not isinstance(text, str) or len(text.strip()) == 0:
        return ""

    # 1. Loại bỏ URLs (không mang thông tin ngữ nghĩa)
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' [URL] ', text)
    text = re.sub(r'www\.(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' [URL] ', text)

    # 2. Loại bỏ email (thông tin liên hệ, không phải nội dung)
    text = re.sub(r'\S+@\S+', ' [EMAIL] ', text)

    # 3. Loại bỏ số điện thoại
    text = re.sub(r'(\+84|0)[0-9]{1,3}[-.\s]?[0-9]{3,4}[-.\s]?[0-9]{3,4}', ' [PHONE] ', text)

    # 4. Chuẩn hóa khoảng trắng (chỉ loại bỏ khoảng trắng THỪA)
    # GIỮ LẠI multiple spaces nếu có ý nghĩa, nhưng thường thì không cần
    text = re.sub(r'\s+', ' ', text)

    # 5. Loại bỏ khoảng trắng đầu cuối
    text = text.strip()

    return text


def preserve_features_clean(text: str) -> str:
    if not isinstance(text, str) or len(text.strip()) == 0:
        return ""

    # Unicode normalization
    text = normalize_unicode(text)

    # Minimal cleaning
    text = minimal_clean_text(text)

    # Final unicode normalization
    text = normalize_unicode(text)

    return text


def get_text_statistics(df: pd.DataFrame, text_column: str = 'NỘI DUNG') -> Dict:
    text_lengths = df[text_column].str.len()
    word_counts = df[text_column].str.split().str.len()

    stats = {
        'char_length': {
            'mean': text_lengths.mean(),
            'median': text_lengths.median(),
            'std': text_lengths.std(),
            'min': text_lengths.min(),
            'max': text_lengths.max(),
            'percentiles': {
                '50%': text_lengths.quantile(0.50),
                '75%': text_lengths.quantile(0.75),
                '90%': text_lengths.quantile(0.90),
                '95%': text_lengths.quantile(0.95),
                '99%': text_lengths.quantile(0.99),
            }
        },
        'word_count': {
            'mean': word_counts.mean(),
            'median': word_counts.median(),
            'min': word_counts.min(),
            'max': word_counts.max(),
        }
    }

    return stats


def analyze_fake_news_features(df: pd.DataFrame, text_col: str, label_col: str) -> Dict:
    fake_df = df[df[label_col] == 0]
    real_df = df[df[label_col] == 1]

    def count_feature(text_series, pattern):
        return text_series.apply(lambda x: len(re.findall(pattern, str(x)))).mean()

    def uppercase_ratio(text_series):
        def calc_ratio(text):
            if not isinstance(text, str) or len(text) == 0:
                return 0
            uppercase = sum(1 for c in text if c.isupper())
            letters = sum(1 for c in text if c.isalpha())
            return uppercase / letters if letters > 0 else 0
        return text_series.apply(calc_ratio).mean()

    emoji_pattern = r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF]'

    features = {
        'fake_news': {
            'avg_exclamation': count_feature(fake_df[text_col], r'!'),
            'avg_question': count_feature(fake_df[text_col], r'\?'),
            'avg_emoji': count_feature(fake_df[text_col], emoji_pattern),
            'uppercase_ratio': uppercase_ratio(fake_df[text_col]),
            'avg_numbers': count_feature(fake_df[text_col], r'\d'),
            'avg_length': fake_df[text_col].str.len().mean()
        },
        'real_news': {
            'avg_exclamation': count_feature(real_df[text_col], r'!'),
            'avg_question': count_feature(real_df[text_col], r'\?'),
            'avg_emoji': count_feature(real_df[text_col], emoji_pattern),
            'uppercase_ratio': uppercase_ratio(real_df[text_col]),
            'avg_numbers': count_feature(real_df[text_col], r'\d'),
            'avg_length': real_df[text_col].str.len().mean()
        }
    }

    return features


def check_data_balance(df: pd.DataFrame, label_column: str = 'GIẢ(0)/THẬT(1)') -> Dict:
    label_counts = df[label_column].value_counts()
    total = len(df)

    balance_info = {
        'total_samples': total,
        'class_distribution': label_counts.to_dict(),
        'class_ratios': (label_counts / total * 100).to_dict(),
        'imbalance_ratio': max(label_counts) / min(label_counts) if len(label_counts) > 1 else 1.0
    }

    return balance_info


In [ ]:
# ============================================================================
# PHẦN 2: PIPELINE TIỀN XỬ LÝ
# ============================================================================

def process_dataset_minimal(
    input_path: str,
    output_path: str,
    text_column: str = 'NỘI DUNG',
    label_column: str = 'GIẢ(0)/THẬT(1)',
    verbose: bool = True
) -> pd.DataFrame:
    if verbose:
        print(f"\n{'='*80}")

    # 1. Đọc dữ liệu
    df = pd.read_csv(input_path)
    initial_size = len(df)

    if verbose:
        print(f"\n THỐNG KÊ BAN ĐẦU:")
        print(f"   Số mẫu: {initial_size}")

    # 2. Xử lý missing values
    df = df.dropna(subset=[text_column, label_column])

    # 3. Loại bỏ duplicates
    df = df.drop_duplicates(subset=[text_column], keep='first')
    df = df.reset_index(drop=True)

    if verbose:
        print(f"   Sau khi loại duplicates: {len(df)} mẫu")

    # 4. PHÂN TÍCH ĐẶC TRƯNG TRƯỚC KHI XỬ LÝ
    if verbose:
        print(f"\n PHÂN TÍCH ĐẶC TRƯNG FAKE NEWS (trước xử lý):")
        features = analyze_fake_news_features(df, text_column, label_column)

        print(f"\n   FAKE NEWS:")
        print(f"   - Trung bình dấu '!': {features['fake_news']['avg_exclamation']:.2f}")
        print(f"   - Trung bình dấu '?': {features['fake_news']['avg_question']:.2f}")
        print(f"   - Trung bình emoji: {features['fake_news']['avg_emoji']:.2f}")
        print(f"   - Tỷ lệ CHỮ HOA: {features['fake_news']['uppercase_ratio']*100:.2f}%")
        print(f"   - Trung bình số: {features['fake_news']['avg_numbers']:.2f}")

        print(f"\n   REAL NEWS:")
        print(f"   - Trung bình dấu '!': {features['real_news']['avg_exclamation']:.2f}")
        print(f"   - Trung bình dấu '?': {features['real_news']['avg_question']:.2f}")
        print(f"   - Trung bình emoji: {features['real_news']['avg_emoji']:.2f}")
        print(f"   - Tỷ lệ CHỮ HOA: {features['real_news']['uppercase_ratio']*100:.2f}%")
        print(f"   - Trung bình số: {features['real_news']['avg_numbers']:.2f}")

    # 5. Áp dụng làm sạch TỐI THIỂU
    if verbose:
        print(f"")

    df['processed_text'] = df[text_column].apply(preserve_features_clean)

    # 6. Loại bỏ văn bản quá ngắn (< 5 ký tự) - threshold thấp hơn
    df = df[df['processed_text'].str.len() >= 5]
    df = df.reset_index(drop=True)

    # 7. PHÂN TÍCH ĐẶC TRƯNG SAU KHI XỬ LÝ
    if verbose:
        print(f"\n PHÂN TÍCH ĐẶC TRƯNG FAKE NEWS (sau xử lý):")
        features_after = analyze_fake_news_features(df, 'processed_text', label_column)

        print(f"\n   FAKE NEWS:")
        print(f"   - Trung bình dấu '!': {features_after['fake_news']['avg_exclamation']:.2f}")
        print(f"   - Trung bình dấu '?': {features_after['fake_news']['avg_question']:.2f}")
        print(f"   - Trung bình emoji: {features_after['fake_news']['avg_emoji']:.2f}")
        print(f"   - Tỷ lệ CHỮ HOA: {features_after['fake_news']['uppercase_ratio']*100:.2f}%")

        # So sánh trước vs sau
        emoji_preserved = (features_after['fake_news']['avg_emoji'] / max(features['fake_news']['avg_emoji'], 0.001)) * 100
        caps_preserved = (features_after['fake_news']['uppercase_ratio'] / max(features['fake_news']['uppercase_ratio'], 0.001)) * 100

        print(f"\n    TỶ LỆ GIỮ LẠI ĐẶC TRƯNG:")
        print(f"   - Emoji: {emoji_preserved:.1f}%")
        print(f"   - Chữ HOA: {caps_preserved:.1f}%")

    # 8. Thống kê văn bản
    stats = get_text_statistics(df, 'processed_text')
    if verbose:
        print(f"\n THỐNG KÊ VĂN BẢN:")
        print(f"   - Độ dài trung bình: {stats['char_length']['mean']:.1f} ký tự")
        print(f"   - Số từ trung bình: {stats['word_count']['mean']:.1f} từ")
        print(f"   - Percentile 95%: {stats['char_length']['percentiles']['95%']:.0f} ký tự")

    # 9. Kiểm tra balance
    balance_info = check_data_balance(df, label_column)
    if verbose:
        print(f"\n  PHÂN BỐ NHÃN:")
        for label, count in balance_info['class_distribution'].items():
            ratio = balance_info['class_ratios'][label]
            label_name = "THẬT" if label == 1 else "GIẢ"
            print(f"   - {label_name} ({label}): {count} ({ratio:.1f}%)")

    # 10. Tạo output DataFrame
    output_df = pd.DataFrame({
        'text': df['processed_text'],
        'label': df[label_column],
        'original_text': df[text_column]
    })

    # 11. Lưu file
    output_df.to_csv(output_path, index=False, encoding='utf-8')

    if verbose:
        print(f"\n ĐÃ LƯU: {output_path}")
        print(f"{'='*80}\n")

    return output_df

In [ ]:
# ============================================================================
# PHẦN 3: HÀM MAIN
# ============================================================================

def main():
    print("\n" + "="*80)
    print("DATA PREPROCESSING")
    print("="*80)

    # Định nghĩa đường dẫn
    data_dir = '/content/sample_data'
    output_dir = '/content/processed_data_minimal'

    # Tạo thư mục output
    import os
    os.makedirs(output_dir, exist_ok=True)

    datasets = {
        'train': {
            'input': f'{data_dir}/train.csv',
            'output': f'{output_dir}/train_processed.csv'
        },
        'val': {
            'input': f'{data_dir}/val.csv',
            'output': f'{output_dir}/val_processed.csv'
        },
        'test': {
            'input': f'{data_dir}/test.csv',
            'output': f'{output_dir}/test_processed.csv'
        }
    }

    # Xử lý từng dataset
    processed_dfs = {}

    for dataset_name, paths in datasets.items():
        try:
            df = process_dataset_minimal(
                input_path=paths['input'],
                output_path=paths['output'],
                verbose=True
            )
            processed_dfs[dataset_name] = df

        except FileNotFoundError:
            print(f"\n Không tìm thấy: {paths['input']}")
            continue
        except Exception as e:
            print(f"\n Lỗi {dataset_name}: {str(e)}")
            continue

    # Tổng kết
    print("\n" + "="*80)
    print("TỔNG KẾT")
    print("="*80)

    for dataset_name, df in processed_dfs.items():
        print(f"\n{dataset_name.upper()}: {len(df)} mẫu")
        print(f"   → {datasets[dataset_name]['output']}")

    # Khuyến nghị
    if 'train' in processed_dfs:
        stats = get_text_statistics(processed_dfs['train'], 'text')
        max_len = int(stats['char_length']['percentiles']['95%'])

    print("HOÀN THÀNH!")
    print("="*80 + "\n")

    return processed_dfs


if __name__ == "__main__":
    processed_data = main()

    # Hiển thị mẫu
    if 'train' in processed_data:
        print("\n" + "="*80)
        print("MẪU DỮ LIỆU (3 dòng đầu):")
        print("="*80)
        sample = processed_data['train'][['text', 'label']].head(3)
        for idx, row in sample.iterrows():
            print(f"\n[{idx+1}] Label: {row['label']} ({'THẬT' if row['label']==1 else 'GIẢ'})")
            print(f"Text: {row['text'][:200]}...")
        print("\n" + "="*80 + "\n")



DATA PREPROCESSING


 THỐNG KÊ BAN ĐẦU:
   Số mẫu: 361
   Sau khi loại duplicates: 220 mẫu

 PHÂN TÍCH ĐẶC TRƯNG FAKE NEWS (trước xử lý):

   FAKE NEWS:
   - Trung bình dấu '!': 0.02
   - Trung bình dấu '?': 0.24
   - Trung bình emoji: 0.00
   - Tỷ lệ CHỮ HOA: 5.63%
   - Trung bình số: 31.14

   REAL NEWS:
   - Trung bình dấu '!': 1.60
   - Trung bình dấu '?': 1.64
   - Trung bình emoji: 0.26
   - Tỷ lệ CHỮ HOA: 3.91%
   - Trung bình số: 14.32


 PHÂN TÍCH ĐẶC TRƯNG FAKE NEWS (sau xử lý):

   FAKE NEWS:
   - Trung bình dấu '!': 0.02
   - Trung bình dấu '?': 0.24
   - Trung bình emoji: 0.00
   - Tỷ lệ CHỮ HOA: 5.64%

    TỶ LỆ GIỮ LẠI ĐẶC TRƯNG:
   - Emoji: 0.0%
   - Chữ HOA: 100.1%

 THỐNG KÊ VĂN BẢN:
   - Độ dài trung bình: 2784.7 ký tự
   - Số từ trung bình: 623.1 từ
   - Percentile 95%: 6342 ký tự

  PHÂN BỐ NHÃN:
   - GIẢ (0): 122 (55.5%)
   - THẬT (1): 98 (44.5%)

 ĐÃ LƯU: /content/processed_data_minimal/train_processed.csv



 THỐNG KÊ BAN ĐẦU:
   Số mẫu: 45
   Sau khi loại dupl